<a href="https://colab.research.google.com/github/saanikagupta/Millet-Disease-Detection-using-Transfer-Learning/blob/master/Millet_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
import numpy as np
from sklearn import metrics
tf.logging.set_verbosity(tf.logging.ERROR)
import os

In [0]:
img_width, img_height = 224, 224
nb_train_samples = 135
nb_validation_samples = 35
batch_size = 5
epochs = 1

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pwd

/content


In [0]:
import shutil
shutil.move("drive/My Drive/plant-leaf.rar", "plant-leaf.rar")
get_ipython().system_raw("unrar x plant-leaf.rar")
shutil.move("plant-leaf.rar", "drive/My Drive/plant-leaf.rar")

'drive/My Drive/plant-leaf.rar'

In [0]:
# get_ipython().system_raw("unrar x plant-leaf")

In [0]:
train_data_dir = 'plant-leaf/train'
test_data_dir = 'plant-leaf/test' 

# VGG16

In [0]:
vgg16 = applications.VGG16(weights = "imagenet", include_top = False, input_shape = (img_width, img_height, 3))

In [0]:
vgg16.layers

In [0]:
# !rm -rf test_aug

In [0]:
# os.mkdir('saved')
train_aug = 'train_aug'
test_aug = 'test_aug'
os.mkdir(train_aug)
os.mkdir(test_aug)

In [0]:
# Freeze the layers which you don't want to train. Here I am freezing the all the layers.
for layer in vgg16.layers[:11]:
    layer.trainable = False

# Adding custom Layers 
x = vgg16.output
x = Flatten()(x)
x = Dense(1024, activation = "relu")(x)
x = Dropout(0.5)(x)
x = Dense(512, activation = "relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation = "softmax")(x)

# creating the final model 
vgg16_final = Model(input = vgg16.input, output = predictions)

# compile the model
sgd = optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
# rmsprop = optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0)
# adagrad = optimizers.Adagrad(lr=0.0001, epsilon=None, decay=0.0)
# adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
vgg16_final.compile(loss = "binary_crossentropy", optimizer = sgd, metrics = ["accuracy"])

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
                rescale = 1./255,
                horizontal_flip = True,
                fill_mode = "nearest",
                zoom_range = 0.3,
                width_shift_range = 0.3,
                height_shift_range = 0.3,
                rotation_range = 30)

train_generator = train_datagen.flow_from_directory(
                  train_data_dir,
                  target_size = (img_height, img_width),
                  batch_size = batch_size, 
                  class_mode = "categorical", shuffle = False,
                  save_to_dir=train_aug, save_prefix='image', save_format='png')

test_datagen = ImageDataGenerator(
                rescale = 1./255,
                horizontal_flip = True,
                fill_mode = "nearest",
                zoom_range = 0.3,
                width_shift_range = 0.3,
                height_shift_range = 0.3,
                rotation_range = 30
                )

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                            target_size = (img_height, img_width),
                                            class_mode = "categorical", shuffle = False,
                                            save_to_dir=test_aug, save_prefix='image', save_format='png')


# Save the model according to the conditions  
checkpoint = ModelCheckpoint('saved/weights.{epoch:02d}-{val_loss:.2f}.h5', monitor = 'val_loss', verbose = 1, save_best_only = True, save_weights_only = False, mode = 'auto', period = 1)
# early = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 10, verbose = 1, mode = 'auto')


# Train the model
# vgg16_final.fit_generator(train_generator,
#                           samples_per_epoch = nb_train_samples,
#                           epochs = epochs,
#                           validation_data = test_generator,
#                           nb_val_samples = nb_validation_samples,
#                           callbacks = [checkpoint, early])

vgg16_final.fit_generator(train_generator,
                          samples_per_epoch = nb_train_samples,
                          epochs = epochs,
                          validation_data = test_generator,
                          nb_val_samples = nb_validation_samples,
                          callbacks = [checkpoint])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


Found 135 images belonging to 2 classes.
Found 35 images belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=1, validation_data=<keras_pre..., callbacks=[<keras.ca..., steps_per_epoch=27, validation_steps=35)`


Epoch 1/1
27/27 [==============================] - 23s 853ms/step - loss: 1.3079 - acc: 0.5037 - val_loss: 2.0708 - val_acc: 0.5997

Epoch 00001: val_loss improved from inf to 2.07079, saving model to saved/weights.01-2.07.h5


In [0]:
# # Loading the trained model
# vgg16_final = load_model('saved/weights.08-1.57.h5')

In [0]:
score, acc = vgg16_final.evaluate_generator(test_generator, verbose = 0)

In [0]:
score

1.5807531127972263

In [0]:
print('Test Accuracy: {}%' .format(acc * 100))

Test Accuracy: 71.42857142857143%


# Confusion Matrix

In [0]:
# Confusion Matrix and Classification Report
Y_pred = vgg16_final.predict_generator(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm = metrics.confusion_matrix(test_generator.classes, y_pred)
print(cm)
acc = 100 * (cm[0][0] + cm[1][1]) / (cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
print('\nAccuracy: ', acc, '%')

Confusion Matrix
[[11  4]
 [ 6 14]]

Accuracy:  71.42857142857143 %


In [0]:
test_generator.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [0]:
y_pred

array([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1])